# Ejemplo de uso de API

## Documentación

[https://linkedin-api.readthedocs.io/en/latest/api.html](https://linkedin-api.readthedocs.io/en/latest/api.html)

### Criterios para busquedas

- Busquedas:
    - Busqueda de empresas:
        - Por nombre 
        - Por sector (Parece ser No viable desde el API)
    - Busqueda personas:
        - Por URL perfil
        - Por sector (Unica que no puede ir sola)
        - Por empresa
        - Por skills o hashtags
        - Por pertenecer a la red de alguien
        - Por locacion
        - Por lenguaje
- Presentacion resultados:
    - Tabulados de forma amena
    - Exportados a Hoja de calculo

### Notas

- Este campo: "urn:li:fs_industry:6" define el sector, pero no se como parsearlo todavia.

- Validar si la distancia funciona como criterio de aceptacion


In [ ]:
from linkedin_api import Linkedin
import json
from configparser import ConfigParser
import pandas as pd
import numpy as np

In [ ]:
parser = ConfigParser()

In [ ]:
parser.read('config.cfg')

In [ ]:
parser.sections()

In [ ]:
credentials = parser['credentials']

In [ ]:
[x for x in credentials.keys()]

In [ ]:
#json_prof

In [ ]:
keywords = '''#statistic
#statistics
#statistician
#bitcoin 
#datascience 
#data 
#stats 
#math 
#mathematics 
#dataanalytics 
#machinelearning 
#analytics 
#python 
#stica 
#cryptoexplorer 
#statistik
#statisticsclass
#statisticschallenge
#statisticsboard
#statisticshelp
#statisticsday
#statisticsandprobability
#statisticsisfun
#statisticsexam
#statisticsteacher
#statisticstutor'''
keywords = keywords.replace("#", "")
keywords = keywords.split("\n")

In [ ]:
industries = '''Fintech
Insurance
Retail
E-commerce 
Finance 
Open Banking'''
industries = industries.split("\n")

In [ ]:
class linkedin_search:
    def __init__(self, user, passw):
        self.user = user
        self.passw = passw
        self.last_query = None
  
    @property
    def api(self):
        return Linkedin(self.user, self.passw)
    
    def search_company(self, name):
        self.company_data = self.api.get_company(name)
        self.last_query = "company"
        return self.company_data
    
    def search_by_profile(self, url_profile):
        self.data_profile = self.api.get_profile(url_profile)
        self.last_query = "profile"
        return self.data_profile
        
    def search_people(self, 
                      keywords=None, 
                      connection_of=None, 
                      current_company=None, 
                      regions=['100876405'], 
                      industries=None,
                      profile_languages=['es'],
                      limit=10):
        
        self.lst_people = self.api.search_people(keywords=keywords, 
                                                 connection_of=connection_of, 
                                                 current_company=current_company, 
                                                 regions=regions, 
                                                 industries=industries, 
                                                 profile_languages=profile_languages,
                                                 limit=limit)
        
        self.lst_single_data = []
        for person in self.lst_people:
            print(f"Get info for urn_id:{person['urn_id']}")
            self.lst_single_data.append(self.api.get_profile(urn_id=person['urn_id']))
        self.last_query = "lst_profiles"
        return {'lst_people': self.lst_people, 'lst_all_data': self.lst_single_data}
    
    def parse_single_profile(self):
        basic_df = pd.DataFrame({'Name': [self.data_profile['firstName'] + ' ' + self.data_profile['lastName']],
                                 'Location': self.data_profile['locationName'],
                                 'Profile_id': self.data_profile['profile_id'],
                                 'Summary': self.data_profile['summary'],
                                 'Actual_Title': self.data_profile['headline'],
                                 'Skills': ', '.join([ x['name'] for x in self.data_profile['skills']])})
        
        experience_df = self.parse_experience(self.data_profile['experience'])
        
        complete_df = pd.merge(basic_df, experience_df, left_index=True, right_index=True, how='outer')
        
        return complete_df
    
    def parse_experience(self,lst_experience):
        df_experience = pd.DataFrame(self.data_profile['experience'])
        columns = ['locationName', 'companyName', 'title', 'timePeriod']
        df_experience = df_experience[columns]
        df_experience.columns = ['Location_Name', 'Company_Name', 'Title', 'Time_Period']
        return df_experience
    
    def parse_many_proiles(self):
        lst_parsed = []
        for index, profile in enumerate(self.lst_single_data):
            print(f'Parse profile number: {index+1}')
            self.data_profile = profile
            lst_parsed.append(self.parse_single_profile())
        return pd.concat(lst_parsed).reset_index(drop=True)
    
    def parse_company_data(self):
        pass
        
    def export_to_dataframe(self):
        if self.last_query == "company":
            return parse_company_data()
        if self.last_query == "profile":
            return self.parse_single_profile()
        if self.last_query == "lst_profiles":
            return self.parse_many_proiles()
        

## 1. Busquedas por perfil

In [ ]:
my_search = linkedin_search(credentials['user'], credentials['pass'])

In [ ]:
data_profile = my_search.search_by_profile('camilo-reyes-02913564')

In [ ]:
data_profile

In [ ]:
df_profile = my_search.export_to_dataframe()
df_profile

In [ ]:
df_profile.to_excel(f'Perfil_{(df_profile.iloc[0]["Name"]).replace(" ","_")}.xlsx', index=False)

## 2. Busquedas por compañia

In [ ]:
data_company = my_search.search_company('Mercadolibre')

In [ ]:
data_company

### 2.1 Validacion mediante herramienta externa

[Json viewer](http://jsonviewer.stack.hu/)

In [ ]:
json.dumps(data_company)

In [ ]:
data_company['entityUrn']

## 3. Busqueda personas bajo palabras clave

In [ ]:
lst_people = my_search.search_people(keywords=['Data', 'Scientist'], limit=10)

In [ ]:
lst_people['lst_people']

In [ ]:
lst_people['lst_all_data']

In [ ]:
df_many_profiles = my_search.export_to_dataframe()

In [ ]:
df_many_profiles.to_excel('Many_profiles.xlsx', index=False)

## 4. Envio de mensajes personalizados

Paara hacer el envio de mensajes debe proporcionarse el campo: ***'entityUrn': 'urn:li:fs_profile:'*** del perdil del usuario 

In [ ]:
api= my_search.api

In [ ]:
api.send_message(recipients=['ACoAAA2LoOEB496sl7nq_lrn3y2Bk7HwhGu8-HQ'], message_body='Prueba mensajes')